In [1]:
import re
import glob
import random, math
from machine_learning import split_data
from collections import defaultdict, Counter
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing

def tokenize(message):
    message = message.lower()
    all_words = re.findall("[a-z0-9]+", message)
    return set(all_words)

def count_words(training_set):
    counts = defaultdict(lambda: [0,0])
    for message, is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] += 1
    return counts

def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    return [(w,
             (spam + k) / (total_spams + 2 * k),
             (non_spam + k) / (total_non_spams + 2 * k))
             for w, (spam, non_spam) in counts.items()]

def spam_probability(word_probs, message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0

    for word, prob_if_spam, prob_if_not_spam in word_probs:
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)

    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

class NaiveBayesClassifier:

    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

    def train(self, training_set):

        num_spams = len([is_spam
                         for message, is_spam in training_set
                         if is_spam])
        num_non_spams = len(training_set) - num_spams

        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts,
                                             num_spams,
                                             num_non_spams,
                                             self.k)

    def classify(self, message):
        return spam_probability(self.word_probs, message)

def p_spam_given_word(word_prob):
    word, prob_if_spam, prob_if_not_spam = word_prob
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [6]:
path = r"C:\\Users\\Dawid\\Desktop\\spam\\*\\*"

data = []

for fn in glob.glob(path):
        is_spam = "ham" not in fn

        with open(fn,'r',encoding='ISO-8859-1') as file:
            for line in file:
                if line.startswith("Subject:"):
                    subject = re.sub(r"^Subject: ","",line).strip()
                    data.append((subject, is_spam))

random.seed(0)
train_data, test_data = split_data(data, 0.6)

classifier = NaiveBayesClassifier()
classifier.train(train_data)

classified = [(subject, is_spam, classifier.classify(subject))
              for subject, is_spam in test_data]

counts = Counter((is_spam, spam_probability > 0.5)
              for _, is_spam, spam_probability in classified)
    
classified.sort(key=lambda row: row[2])
    
spammiest_hams = list(filter(lambda row: not row[1], classified))[-5:]
hammiest_spams = list(filter(lambda row: row[1], classified))[:5]


words = sorted(classifier.word_probs, key=p_spam_given_word)

spammiest_words = words[-5:]
hammiest_words = words[:5]

print("spammiest_words", spammiest_words)
print('\n\n')
print("hammiest_words", hammiest_words)

spammiest_words [('yyyy', 0.015257192676547515, 0.00019387359441644047), ('95', 0.016129032258064516, 0.00019387359441644047), ('credit', 0.02048823016564952, 0.00019387359441644047), ('mortgage', 0.03443766346992153, 0.00019387359441644047), ('adv', 0.03879686137750654, 0.00019387359441644047)]



hammiest_words [('users', 0.00043591979075850045, 0.05951919348584723), ('razor', 0.00043591979075850045, 0.057192710352849944), ('satalk', 0.00043591979075850045, 0.04439705312136487), ('spambayes', 0.00043591979075850045, 0.03392787902287708), ('zzzzteana', 0.00043591979075850045, 0.0335401318340442)]
